In [12]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [13]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

# Creating The Neural Network


In [14]:
class NeuralNetwork(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
    super(NeuralNetwork, self).__init__()
    self.seed = torch.manual_seed(seed)
    # nn.Linear(number of the inputs in the first layer, number of the neurons in the first layer)
    # Linear : y = W*x + b
    self.fc1 = nn.Linear(state_size, 64)
    # this takes the 64 outputs from the layer before and take them as input
    self.fc2 = nn.Linear(64, 64)
    # here the output is actually 4 actions as described in gymnasium website about the lunar landing, 0 -> do nothing
    # 1 -> go left, 2-> main engine, 3-> go right
    self.fc3 = nn.Linear(64, action_size)

  # forward propogation from the input layer to the output layer
  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

# Training The Module


## Set Up


In [15]:
import gymnasium as gym
env = gym.make("LunarLander-v3")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [16]:
print(f"State Shape: {state_shape}")
print(f"State Size: {state_size}")
print(f"Num of Actions: {action_size}")

State Shape: (8,)
State Size: 8
Num of Actions: 4


## Init the hyperparams

In [17]:
learning_rate = 5e-4 # 0.0005
min_batch_size = 100
# gamma defines how far the agent looks into the future when deciding on an action
#
gamma = 0.99 # discount factor
replay_experience_size = int(1e5)
tau = 1e-3 # for soft update of target parameters (interpolation parameter)

## Implementing the Experience Replay

In [18]:
class ReplayMemory(object):
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    # experience = (state,action,reward,next_state,done)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones


## DQN Implementation

In [19]:
class Agent():

  def __init__(self,state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
    self.target_qnetwork = NeuralNetwork(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_experience_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1)%4 # when its step 4 its actually 0
    if self.t_step == 0:
      if len(self.memory.memory) > min_batch_size:
        experiences = self.memory.sample(min_batch_size)
        self.learn(experiences, gamma)

  # epsilon greedy algorithm
  # Higher epsilon means more exploration (random actions),
  # and lower epsilon means more exploitation (choosing the action with the highest Q-value).
  def act(self, state, eps = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    # the epsilon greedy part
    if random.random() > eps:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, gamma):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (gamma * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, tau)

  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


## Init the Agent

In [20]:
agent = Agent(state_size, action_size)

## Train the Agent

In [24]:
num_episodes = 2000
max_num_timesteps_per_ep = 1000
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay_rate = 0.995
epsilon = epsilon_start
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, num_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(max_num_timesteps_per_ep):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_end, epsilon_decay_rate*epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 220.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break




Episode 100	Average Score: -102.48
Episode 200	Average Score: -31.27
Episode 300	Average Score: 62.30
Episode 400	Average Score: 155.60
Episode 495	Average Score: 223.55
Environment solved in 395 episodes!	Average Score: 223.55


# Visualizing the results

In [25]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()